In [3]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

In [14]:
# to understand imputation
# a = pd.DataFrame([
#                 [2, 6],
#                 [5,3],
#                 [6, np.nan],
#                 [np.nan, 8]
#                  ])
# b = pd.DataFrame([
#                 [np.nan, 1],
#                 [3,4],
#                 [8, 1],
#                 [3, np.nan]
#                  ])
# sim_imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
# print(type(a))
# transformed_a = pd.DataFrame(sim_imp.fit_transform(a))
# print(type(transformed_a))
# sim_imp.transform(b)


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


array([[4.33333333, 1.        ],
       [3.        , 4.        ],
       [8.        , 1.        ],
       [3.        , 5.66666667]])

In [27]:
# Load melbourne housing data
data = pd.read_csv('../../datasets/melb_data.csv')
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [32]:

#pick target column
y = data['Price']

# drop target column from data frame
melb_predictors = data.drop(['Price'], axis = 1)

# pick feature columns 
X = melb_predictors.select_dtypes(exclude=['object'])
# for simplicity, will pick only columns which have numerical values
X.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,144.9969,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,4019.0


In [61]:
#train and test data split
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)
print(X_train.shape)
print(X_valid.shape)

(10864, 12)
(2716, 12)


In [52]:
#list columns which has missing values
missing_value_columns = [col for col in X_valid.columns if X_valid[col].isnull().any()]
missing_value_columns

['Car', 'BuildingArea', 'YearBuilt']

In [62]:
#simple Imputation
simple_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
trans_X_train = pd.DataFrame(simple_imputer.fit_transform(X_train))
trans_X_valid = pd.DataFrame(simple_imputer.transform(X_valid))

#After imputation column names(i.e headers) were removed.put them back
trans_X_train.columns = X_train.columns
trans_X_valid.columns = X_valid.columns

In [67]:
#function to calculate error
#score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100,random_state=0) #criterion="mae",
    model.fit(X_train, y_train)
    y_valid_prediction = model.predict(X_valid)
    return mean_absolute_error(y_valid, y_valid_prediction) 

In [68]:
score = score_dataset(trans_X_train, trans_X_valid, y_train, y_valid)
#y_train.head()
score

171893.16509607964